Техническое задание на автоматизацию отслеживания оплат аренды гаражей и уведомления о просрочках

**Общие требования:**

Автоматизированная система должна отслеживать своевременность оплаты аренды за гаражи и указывать статус платежа

**Исходные данные:**

* Используется Google-таблица с колонками:

* Название гаража
* Дата оплаты
* Сумма оплаты (уникальная для каждого гаража)
* Имя арендатора




**1. Способ получения данных об оплатах:**
Информация о поступлениях (в том числе, но не исключительно, по оплатам за аренду гаража) находится в файле print2 c выпиской Сбербанка. Форма выписки может меняться, содержание остается примерно таким же

**2. Идентификация платежа:**
* Оплата идентифицируется по уникальной сумме, указанной в файле Arenda

**3. Отслеживание сроков оплаты:**

* При нажатии на иконку приложения/кнопку внутри одной из таблиц/прочие варианты должна по запросу проверять даты оплаты согласно таблице.
* В случае, если дата оплаты приходится на 31 число (29, 30е для февраля), а месяц имеет короче дня, датой оплаты считать последний день месяца.

**4. Условия уведомлений о просроченных платежах:**

* Если оплата не поступила в течение 3 календарных дней после установленной даты, платеж считается просроченным

**5. Формат уведомления:**

По итогам проверки должен создаваться xlsx-файл с указанием статуса платежа (получен / просрочен / срок не наступил)
* Уведомление должно содержать следующую информацию:

* Название гаража
* Дата оплаты (ожидаемая)
* Сумма оплаты
* Статус

Сервис должен работать и при изменении количества гаражей, их названий, сумм, формы предоставления информации в выписке

In [ ]:
pip install pandas openpyxl numpy

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
import calendar
import re

# ==== НАСТРОЙКИ ====
TODAY = datetime.today().date()  # можно зафиксировать дату: datetime(2025, 8, 11).date()
ARENDA_FILE = "/content/arenda.xlsx"
BANK_FILE = "/content/print 2.xlsx"
REPORT_FILE = "/content/report.xlsx"

# ==== ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ====
def parse_amount(x):
    """Преобразует суммы к числу (float), удаляя символы валют и форматирование."""
    if pd.isna(x):
        return None
    if isinstance(x, (int, float, np.number)):
        return float(x)
    s = str(x).strip()
    s = re.sub(r'[^\d\-,\.]', '', s)
    if s.count(',') > 0 and s.count('.') == 0:
        s = s.replace(',', '.')
    else:
        s = s.replace(',', '')
    try:
        return float(s)
    except:
        return None

def normalize_columns(df):
    """Сопоставляет нижний регистр имени колонки с оригинальным."""
    return {c.lower().strip(): c for c in df.columns}

def find_col_substring(df_map, substrings):
    """Находит первую колонку, содержащую подстроку из списка."""
    for key, orig in df_map.items():
        for s in substrings:
            if s in key:
                return orig
    return None

def extract_day(x):
    """Извлекает день месяца из даты или строки."""
    if pd.isna(x):
        return None
    if isinstance(x, (int, float, np.integer, np.floating)):
        return int(x)
    try:
        dt = pd.to_datetime(x, dayfirst=True, errors='coerce')
        if pd.notna(dt):
            return int(dt.day)
    except:
        pass
    m = re.search(r'(\d{1,2})', str(x))
    if m:
        return int(m.group(1))
    return None

def expected_date_for_day(day, ref_date):
    """Возвращает дату в месяце ref_date с учётом длины месяца."""
    if day is None:
        return None
    year = ref_date.year
    month = ref_date.month
    last = calendar.monthrange(year, month)[1]
    return datetime(year, month, min(day, last)).date()

def extract_date_from_text(x):
    """Пытается найти дату в тексте."""
    if pd.isna(x):
        return None
    s = str(x)
    m = re.search(r'(\d{1,2}[.\-/]\d{1,2}[.\-/]\d{2,4})', s)
    if m:
        try:
            return pd.to_datetime(m.group(1), dayfirst=True, errors='coerce').date()
        except:
            return None
    return None

# ==== ЧТЕНИЕ ДАННЫХ ====
arenda = pd.read_excel(ARENDA_FILE, sheet_name=0)
bank = pd.read_excel(BANK_FILE, sheet_name=0)

# --- Обработка arenda.xlsx ---
a_map = normalize_columns(arenda)
garage_col = find_col_substring(a_map, ["гараж", "наимен", "назван"])
date_col = find_col_substring(a_map, ["дата", "первонач", "срок", "ожидаем"])
amount_col = find_col_substring(a_map, ["сумм", "оплат", "amount"])
tenant_col = find_col_substring(a_map, ["аренд", "имя", "фамил", "tenant"])

if not garage_col or not amount_col or not date_col:
    raise ValueError("Не найдены нужные колонки в arenda.xlsx")

ar = arenda[[garage_col, date_col, amount_col] + ([tenant_col] if tenant_col else [])].copy()
ar.columns = ["garage", "expected_date_raw", "amount"] + (["tenant"] if tenant_col else [])
ar["amount_parsed"] = ar["amount"].apply(parse_amount)
ar["expected_day"] = ar["expected_date_raw"].apply(extract_day)
ar["expected_date"] = ar["expected_day"].apply(lambda d: expected_date_for_day(d, TODAY))

# --- Обработка выписки ---
b_map = normalize_columns(bank)
b_amount_col = find_col_substring(b_map, ["сумм", "amount", "кредит", "зачисл"])
b_date_col = find_col_substring(b_map, ["дата", "date", "операц", "платеж"])
b_info_col = find_col_substring(b_map, ["назнач", "описан", "коммент", "info"])

# fallback для суммы
if not b_amount_col:
    scores = {c: bank[c].apply(parse_amount).notna().sum() for c in bank.columns}
    b_amount_col = max(scores, key=scores.get)
# fallback для даты
if not b_date_col:
    date_scores = {c: pd.to_datetime(bank[c], dayfirst=True, errors='coerce').notna().sum() for c in bank.columns}
    b_date_col = max(date_scores, key=date_scores.get)

bank_norm = bank[[b_amount_col, b_date_col] + ([b_info_col] if b_info_col else [])].copy()
bank_norm.columns = ["amount_raw", "date_raw"] + (["info"] if b_info_col else [])
bank_norm["amount_parsed"] = bank_norm["amount_raw"].apply(parse_amount)
bank_norm["date_parsed"] = pd.to_datetime(bank_norm["date_raw"], dayfirst=True, errors='coerce').dt.date

mask = bank_norm["date_parsed"].isna()
if "info" in bank_norm.columns:
    bank_norm.loc[mask, "date_parsed"] = bank_norm.loc[mask, "info"].apply(extract_date_from_text)

# ==== СВЕРКА ====
results = []
for _, row in ar.iterrows():
    garage = row["garage"]
    expected_date = row["expected_date"]
    amount = row["amount_parsed"]

    match = None
    if amount is not None:
        tol = 1e-2
        candidates = bank_norm[bank_norm["amount_parsed"].notna() &
                               (bank_norm["amount_parsed"].sub(amount).abs() <= tol)]
        if not candidates.empty:
            match = candidates.sort_values("date_parsed").iloc[0]

    payment_date = match["date_parsed"] if isinstance(match, pd.Series) else None

    if payment_date is not None:
        status = "получен"
    else:
        if expected_date and TODAY > (expected_date + timedelta(days=3)):
            status = "просрочен"
        else:
            status = "срок не наступил"

    results.append({
        "Название гаража": garage,
        "Ожидаемая дата оплаты": expected_date,
        "Сумма оплаты": amount,
        "Статус": status,
        "Дата платежа (из выписки)": payment_date
    })

# ==== ВЫВОД ====
report = pd.DataFrame(results)
report.to_excel(REPORT_FILE, index=False)
print(f"Отчёт сохранён в файл: {REPORT_FILE}")
print(report)


Отчёт сохранён в файл: /content/report.xlsx
    Название гаража Ожидаемая дата оплаты  Сумма оплаты            Статус  \
0                 1            2025-08-01        4295.0         просрочен   
1                 2            2025-08-01        3301.0         просрочен   
2                 3            2025-08-01        2797.0         просрочен   
3                 4            2025-08-02        2495.0         просрочен   
4                 5            2025-08-03        3250.0         просрочен   
5                 6            2025-08-05        3500.0         просрочен   
6                 7            2025-08-05        3950.0         просрочен   
7                 8            2025-08-05        4750.0         просрочен   
8                 9            2025-08-18        2600.0  срок не наступил   
9                10            2025-08-07        2990.0         просрочен   
10               11            2025-08-22        2500.0  срок не наступил   
11               12            2

/tmp/ipython-input-2700241234.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_scores = {c: pd.to_datetime(bank[c], dayfirst=True, errors='coerce').notna().sum() for c in bank.columns}
/tmp/ipython-input-2700241234.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_scores = {c: pd.to_datetime(bank[c], dayfirst=True, errors='coerce').notna().sum() for c in bank.columns}
/tmp/ipython-input-2700241234.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_scores = {c: pd.to_datetime(bank[c], dayfirst=True, errors='coerce').notna().sum() for c in bank.colu